<a href="https://colab.research.google.com/github/itissandeep98/StackOverFlow-Tag-Predictor/blob/master/MLproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [29]:
import numpy as np
import pandas as pd 
import csv,nltk,random,pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
import pickle
import joblib
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Offline Preprocessing

In [ ]:
af=pd.read_csv('/content/drive/MyDrive/Ml Project/Train.zip')
df=af[:1000000]

duplicate_pairs = df.sort_values('Title', ascending=False).duplicated('Title')
df = df[~duplicate_pairs]

df["Tags"]=df["Tags"].astype(str)
df["tag_count"] = df["Tags"].apply(lambda x : len(x.split()))

In [ ]:
# Sandeep Dont Delete This 
df.head()

In [ ]:
qus_list=[]

for index,row in df.iterrows():
    title, body, tags = row["Title"], row["Body"], row["Tags"]   
    body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
    body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
    title=title.encode('utf-8')
    question=str(title)+" "+str(body)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    qus_list.append(question)
   
df["question"] = qus_list

In [ ]:
# Sandeep Dont delete This
preprocessed_df = df[["question","Tags"]]
preprocessed_df

In [ ]:
preprocessed_df.to_csv("preprocessed_20k.csv")

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Ml Project/preprocessed_file.csv')
preprocessed_df = df[["question","Tags"]]

In [ ]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
multilabel_y = vectorizer.fit_transform(preprocessed_df['Tags'])

In [ ]:
def tags_to_consider(n):
    tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=multilabel_y[:,sorted_tags_i[:n]]
    #names=[vectorizer.get_feature_names() for i in multilabel_y[:,sorted_tags_i[:n]] ]
    return yn_multilabel

def questions_explained_fn(n):
    multilabel_yn = tags_to_consider(n)
    x= multilabel_yn.sum(axis=1)
    return (np.count_nonzero(x==0))

In [ ]:
questions_explained = []
total_tags=multilabel_y.shape[1]
total_qs=preprocessed_df.shape[0]
for i in range(500, total_tags, 100):
    questions_explained.append(np.round(((total_qs-questions_explained_fn(i))/total_qs)*100,3))


In [ ]:
yx_multilabel = tags_to_consider(5500)
#print("number of questions that are not covered :", questions_explained_fn(5500),"out of ", total_qs)
#print("Questions covered ", (total_qs -questions_explained_fn(5500) )*100 / total_qs )

In [ ]:
names=vectorizer.get_feature_names()
tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
ans=[]
for i in sorted_tags_i:
    ans.append(names[i])
np.savetxt("tag_names.csv",ans, delimiter =", ",  fmt ='% s') 
print(ans)

# Spliting Data into training and testing


In [3]:
df=pd.read_csv('/content/drive/MyDrive/Ml Project/preprocessed_60k.csv')
preprocessed_df = df[["question","Tags"]]
preprocessed_df

,question,Tags
0,check upload file imag without mime type like ...,php image-processing file-upload upload mime-t...
1,prevent firefox close press ctrl favorit edito...,firefox
2,error invalid type list variabl import matlab ...,r matlab machine-learning
3,replac special charact url probabl simpl simpl...,c# url encoding
4,modifi whoi contact detail use modifi function...,php api file-get-contents
...,...,...
59768,compil run java one folder follow directori st...,java
59769,creat paramet function clang sourc code look l...,llvm clang
59770,eclips subvers launch configur cynosur refer n...,eclipse svn subversive
59771,learn master visual studio good resourc book l...,learning books visual-studio resources visual-...


In [4]:
preprocessed_df["tag_count"] = preprocessed_df["Tags"].apply(lambda text: len(text.split(" ")))

In [5]:
preprocessed_df.tag_count.value_counts()

3    17267
2    15861
4    11414
1     8218
5     7013
Name: tag_count, dtype: int64

In [6]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
multilabel_y = vectorizer.fit_transform(preprocessed_df['Tags'])

In [7]:
a=multilabel_y.sum(axis=0).tolist()[0]
print(pd.DataFrame(a).value_counts(),len(a))

1       6454
2       2444
3       1272
4        765
5        610
        ... 
215        1
214        1
212        1
91         1
4623       1
Length: 255, dtype: int64 15044


In [8]:
print(multilabel_y.shape)
tags=vectorizer.get_feature_names()
print(tags)

(59773, 15044)
['.class-file', '.each', '.emf', '.hgtags', '.htaccess', '.htpasswd', '.mov', '.net', '.net-1.1', '.net-2.0', '.net-3.5', '.net-4.0', '.net-4.5', '.net-assembly', '.net-cf-3.5', '.net-framework', '.net-micro-framework', '.net-remoting', '.net2.0', '.net4.0', '.nettiers', '.obj', '.post', '.refresh', '.war', '.x', '1080p', '10gbethernet', '10gige', '16bit', '1and1', '2003', '2007', '2010', '2013', '24bit', '24hour', '2d', '2d-array', '2d-games', '2wire', '3-tier', '3.0', '3.0.1', '301-redirect', '32-bit', '32bit', '32bit-64bit', '37-signals', '3d', '3d-engine', '3d-model', '3d-modelling', '3des', '3dsmax', '3g', '3rd-party-library', '3ware', '401', '403', '404', '404-error', '4d', '5.1', '500-error', '503-error', '64-bit', '64bit', '6502', '6510', '68000', '7-zip', '70-667', '7zip', '802.11', '802.11n', '8086', '8bit', '9-bit-serial', '9p', 'a-record', 'a-star', 'aabb', 'aac', 'aapt', 'abaddressbook', 'abap', 'abc', 'abelian-groups', 'aberration', 'abi', 'abpersonviewcont

In [9]:
def tags_to_consider(n):
    tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=multilabel_y[:,sorted_tags_i[:n]]
    #names=[vectorizer.get_feature_names() for i in multilabel_y[:,sorted_tags_i[:n]] ]
    return yn_multilabel
yx_multilabel = tags_to_consider(500)
yx_multilabel.todense()

matrix([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [10]:
a=yx_multilabel.sum(axis=0).tolist()[0]
print(pd.DataFrame(a).value_counts(),len(a))

54      16
55      14
53      13
57      13
52      11
        ..
134      1
265      1
261      1
255      1
4623     1
Length: 205, dtype: int64 500


In [12]:
# X_train, X_test, y_train, y_test = train_test_split(preprocessed_df, yx_multilabel,stratify=yx_multilabel.todense(), test_size = 0.2,random_state = 42, )
X_train=preprocessed_df.head(47818)
X_test=preprocessed_df.tail(preprocessed_df.shape[0] - 47818)

Y_train = yx_multilabel[0:47818,:]
Y_test = yx_multilabel[47818:preprocessed_df.shape[0],:]


In [13]:
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(47818, 3) (47818, 500) (11955, 3) (11955, 500)


In [14]:
vectorizer = CountVectorizer(min_df=0.00009, max_features=13000,  \
                             tokenizer = lambda x: x.split(), ngram_range=(1,4))
x_train_multilabel = vectorizer.fit_transform(X_train['question'])
x_test_multilabel = vectorizer.transform(X_test['question'])

In [16]:
print(x_train_multilabel.shape,x_test_multilabel.shape)
joblib.dump(x_train_multilabel, 'x_train_multilabel.pkl') 

joblib.dump(x_test_multilabel, 'x_test_multilabel.pkl')

joblib.dump(Y_train, 'y_train.pkl')

joblib.dump(Y_test, 'y_test.pkl')

(47818, 13000) (11955, 13000)


['y_test.pkl']

# Data loading

In [ ]:
rs = ShuffleSplit(n_splits=1, test_size=.3, random_state=0)
train_indices, test_indices = next(rs.split(preprocessed_df['question'].values))
train_indices, test_indices

In [ ]:
X_train, y_train = preprocessed_df.values[train_indices], yx_multilabel[train_indices]
X_test, y_test = preprocessed_df.values[test_indices], yx_multilabel[test_indices]
print(X_train.shape,X_test.shape)

In [ ]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(X_train[:,0])
x_test_multilabel = vectorizer.transform(X_test[:,0])
print(x_train_multilabel.shape)

In [ ]:
x_train_multilabel.todense()

# Logistic Regression

In [23]:
# param={'estimator__alpha': [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1]}
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.001, penalty='l1'))
# gsv = GridSearchCV(estimator = clf, param_grid=param, cv=3, verbose=0, scoring='f1_micro',n_jobs=-1)
clf.fit(x_train_multilabel, Y_train)
pickle.dump(clf,open('logistic1.sav','wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [25]:
y_pred = clf.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred, average = 'micro'))

Accuracy : 0.18293601003764115
Macro f1 score : 0.21628560875515915
Micro f1 scoore : 0.3783643724696356


# SVM

In [21]:
clf_svm = OneVsRestClassifier(SGDClassifier(alpha=0.001, penalty='l2'))
clf_svm.fit(x_train_multilabel, Y_train)
pickle.dump(clf_svm,open('svm.sav','wb'))

In [22]:
y_pred_svm = clf_svm.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred_svm))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred_svm, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred_svm, average = 'micro'))

Accuracy : 0.22492680886658303
Macro f1 score : 0.24646593537615002
Micro f1 scoore : 0.4239463666071489


# Naive Bayes

In [17]:
clf_nvb = OneVsRestClassifier(MultinomialNB())
clf_nvb.fit(x_train_multilabel, Y_train)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True),
                    n_jobs=None)

In [20]:
y_pred_nvb = clf_nvb.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred_nvb))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred_nvb, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred_nvb, average = 'micro'))

Accuracy : 0.05211208699289
Macro f1 score : 0.19189051237252772
Micro f1 scoore : 0.3098957318894213


# RANDOM FOREST


In [33]:
clf_rf = RandomForestClassifier()
clf_rf.fit(x_train_multilabel, Y_train)

ValueError: ignored

In [ ]:
y_pred_rf = clf_rf.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred_rf))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred_rf, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred_rf, average = 'micro'))

# GaussianProcessClassifier

In [32]:
clf_gpc = OneVsRestClassifier(GaussianProcessClassifier(multi_class='one-vs-one'))
clf_gpc.fit(x_train_multilabel, Y_train)

TypeError: ignored

In [ ]:
y_pred_ = clf_rf.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred_rf))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred_rf, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred_rf, average = 'micro'))